In [ ]:
from mlrepricer import match, setup, helper, schemas, minmax
import altair as alt
import numpy as np
import pandas as pd
import warnings
import datetime as dt
import matplotlib.pyplot as plt


from mlrepricer.database import SQLite

alt.data_transformers.enable('default', max_rows=100000000)

In [5]:
t = schemas.pricemonitor(SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')
filter1 = helper.cleanup(df)
main = helper.normalize(filter1)
main = main.reset_index().set_index(['asin', 'sellerid'])
main = pd.get_dummies(main)

In [6]:
# unrelated fast try out of maxmin helper columns
minmaxdf = pd.DataFrame()
minmaxdf['mean'] = main[main.isbuyboxwinner==1].groupby(['asin', 'shipping_time_0'])['price'].mean().round(2)
minmaxdf['min'] = main[main.isbuyboxwinner==1].groupby(['asin', 'shipping_time_0'])['price'].min().round(2)
minmaxdf['max'] = main[main.isbuyboxwinner==1].groupby(['asin', 'shipping_time_0'])['price'].max().round(2)

In [7]:
minmaxdf = minmaxdf.reset_index()
minmaxdf.asin = minmaxdf.asin + np.where(minmaxdf.shipping_time_0 == 0, '_seller', '_prime')
del minmaxdf['shipping_time_0']

In [15]:
old = minmax.load_csv()
old = old.reset_index()
old.asin = old.asin + np.where(old.isprime == 0, '_seller', '_prime')
del old['isprime']

In [20]:
result = pd.merge(minmaxdf, old, how='inner', on='asin')
result.drop(['index', 'maxPrice', 'minPrice'], axis=1).to_csv('minmax.csv')